<a href="https://www.kaggle.com/code/bencaiello/fast-food-data-with-the-new-version-2-of-dataset?scriptVersionId=143137297" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# data wrangling imports:
import pandas as pd 
import numpy as np

# visualization imports:
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning imports:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

#Remove irritating warnings from output
import warnings
warnings.simplefilter('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# First Look
Now that the standard Kaggle import is done, let's open the file and look at it a bit

Note that is it a modified/updated version of my "Fast Food with heavy cleanup" notebook, using the new filepath & removing some of the cleanup steps that the prior version of the dataset required. If you want to see what the problems with the older version of the dataset were & how I fixed them, you can go to that notebook and/or see the discussion topic about this for this dataset.

In [ ]:
# load file
file = pd.read_csv('/kaggle/input/fast-food/FastFoodNutritionMenuV2.csv')

# exploratory examination of file
display(file.head())
file.info()
# file.isna().sum()

# Converting columns to numeric: checking what doesn't fit
All columns except Company / Item should be numeric (I will use float type) but are currently object type, so we will need to eventually convert them to float.

At the same time, I also want to know what is in my columns that aren't numeric, so I wrote a function that will reveal what kinds of values cannot be converted to float in each of these columns, so that I can see what other cleaning steps might be used to salvage data / determine whether other errors may be present:

In [ ]:
def df_non_numeric_list(dataframe, column_exclusion_list = None):
    """ Print all unique values in a dataframe, by column, that cannot be converted to a float.
    
    Parameters
    ----------
    dataframe (pandas dataframe): required, the dataframe whose columns are to be analyzed for non-numeric convertible values
    column_exclusion_list (list): optional, a list of column names of those columns to be excluded from the analysis. Default value is None.
    
    Returns
    -------
    Prints: the name of each column in the dataframe, followed by a list of all the non-numeric convertible values in the column.
    """
    for column in dataframe.columns:
        error_list = []
        if column in column_exclusion_list:
            pass
        else:
            for element in dataframe[column]:
                try:
                    element_float = float(element)
                except:
                    error_list.append(element)
            print(column , ': ', pd.Series(error_list).unique())

# Check what non-numerics are hiding in the columns that ought to be numeric....
df_non_numeric_list(file,['Company', 'Item'])


# Fixing non-numeric values in the numerical columns
I found that the values in a non numeric-convertable form were either:
  
  
  '\xa0', 
 
 
 '<1' or '<5', 
 
 
 plus one instance where the units were included with a usable number ('5.5 g')

In [ ]:
# Simple cleaning
file = file.replace(['\xa0', '<1', '<5', '5.5 g'],[np.nan, '0', '0', '5.5'])

# Confirm that the '<1','<5', and '5.5 g' are no longer in the dataframe using my previously defined function:
print('After fixing a number of the non-convertible items in the numeric columns: re-test to confirm they have been fixed: \n')
df_non_numeric_list(file,['Company', 'Item'])

# convert to float
file_no_company_item = file.drop(['Company', 'Item'], axis = 1)
f_cols = file_no_company_item.columns
for i in f_cols:
    file = file.astype({i : 'float'})

# check for duplicates, and remove (there are 7 duplicates, just like I saw another notebook find)
print(len(file))
file = file.drop_duplicates()
print(len(file))

# remove all rows where every value is NaN (except company/item). 
file = file.dropna(subset = file_no_company_item.columns, how = 'all')
print("\n I've also removed some of the NaN's, but some remain")


# Addressing the remaining Null values in the DataFrame
Now I will drop all the NaN's (there are relatively few, less than 5% of the rows in the dataframe) except the NaN's in the Weight Watchers Points column, which I'll leave as is, and the NaN's in the Calories from Fat column, which I will impute using the approximate of 9 calories / gram of fat. 

The Weight Watchers column, from what I have seen, is exclusively NaN for some companies (so I didn't want to impute a mean WW Points to those), but as we'll see later, the WW Points column does not convey much useful information beyond the information already captured by the Total Calories column of the dataframe (the two values correlate at 0.99!). 

In [ ]:
''' 
First, I will drop all the rows with NaNs below 5% of the columns (which is ~56, from 0.05*1126) then fill all the NaN's of the calories from fat with 9*total fat
Finally I will choose to not impute the NaN's in the Weight Watchers column, as three of the companies simply have no data for that
'''

cols_to_drop_NaN = ['Total Fat\n(g)', 'Saturated Fat\n(g)', 'Trans Fat\n(g)', 'Carbs\n(g)', 'Fiber\n(g)', 'Protein\n(g)']
file = file.dropna(subset = cols_to_drop_NaN)
file['Calories from\nFat'] = file['Calories from\nFat'].fillna(9 * file['Total Fat\n(g)'])
print('Have dropped all nulls except those in the Weight Watcher Points column: \n')
print(file.isna().sum())

# I can see the changes have taken effect -- now it time to finally plot something / move past data cleaning!

# First Look at data: distribution of numerical variables

In [ ]:
# Distributions of all the numerical data in the dataset

fig, ax = plt.subplots(nrows = 3, ncols = 4, sharey = True)
col = 2
for axs in ax.flat:
    axs.hist(file.iloc[:,col])
    axs.set_title(file.columns[col].replace('\n', ' ').replace('Weight Watchers', 'WW'),size = 9, y = 0.7)
    axs.set_ylim(top = 1000)
    col += 1
fig.suptitle("Distributions of Numerical Variables in Dataset")
plt.show()

# boxplot
fileWWdrop = file.dropna() 
fig, ax = plt.subplots(nrows = 3, ncols = 4, sharey = True)
col = 2
for axs in ax.flat:
    if col == 13:
        # Must use a version of the dataframe without Nll values, else the WW points won't plot
        axs.boxplot(fileWWdrop.iloc[:,col])
    else:
        axs.boxplot(file.iloc[:,col])
    axs.set_title(file.columns[col].replace('\n', ' ').replace('Weight Watchers', 'WW'), size = 9, y = 1.0)
    axs.set_ylim(top = 3000)
    axs.set_xticklabels('')
    col += 1
fig.suptitle("Distributions of Numerical Variables in Dataset")
plt.show()

# Potentially useful Transformation: cubed-root

With the except of the total calories and WW pnts columns, the distributions visible above are very right-skewed and have lots of values piled up near 0 with large tails of higher-value outliers, so a transformation of the data may be useful. 

I tried a log-transformation first, but settled on a root transformation because the many zero values in the dataset (even with the addition of a small number like 0.0001 to avoid infinities) seemed to throw off the output for some of the downstream analysis I was doing. A cubed-root ended up being simpler to execute and appeared to convert the data to relatively normal distributions for most columns, except for the pile of values remaining at 0 in many of them:

In [ ]:
# Most of these are heavily right-skewed distributions, so let's transform them by taking the cubed root and see what happens:
file_re_trans = pd.DataFrame()
for i in file_no_company_item.columns:
    file_re_trans[i] = (file[i]) ** (1/3)

# Distributions of all the numerical data in the transformed dataset

fig, ax = plt.subplots(nrows = 3, ncols = 4, sharey = True)
col = 0
for axs in ax.flat:
    axs.hist(file_re_trans.iloc[:, col])
    axs.set_title(file_re_trans.columns[col].replace('\n', ' ').replace('Weight Watchers', 'WW'),size = 9, y = 0.7)
    axs.set_ylim(top = 1000)
    col += 1
fig.suptitle("Distributions of Numerical Variables in Dataset: cubed-root transformation")
plt.show()

# boxplot
filere_WWdrop = file_re_trans.dropna() 
fig, ax = plt.subplots(nrows = 3, ncols = 4, sharey = True)
col = 0
for axs in ax.flat:
    if col == 11:
        axs.boxplot(filere_WWdrop.iloc[:, col])
    else:
        axs.boxplot(file_re_trans.iloc[:, col])    
    axs.set_title(file_re_trans.columns[col].replace('\n', ' ').replace('Weight Watchers', 'WW'), size = 9, y = 1.0)
    axs.set_ylim(top = (3000 ** (1/3)))
    axs.set_xticklabels('')
    col += 1
fig.suptitle("Distributions of Numerical Variables in Dataset: cubed-root transformation")
plt.show()

# Much better! If called for, we can always use a cubed-root transformation, but the transformed dataframe currently doesn't have the first two columns: 
file_re_trans['Company'] = file['Company']
file_re_trans['Item'] = file['Item'] 

# Distributions of calories, sodium, and fiber by company

In [ ]:
# Although there are still some outliers in the data, I want to move on to visualization and analysis for now 

# The question is -- what do we what to know?
# I see other have split things up by company, so I'll look at that too
file['Company'].value_counts().plot(kind = 'bar')
plt.title("Items in the Dataset from each Company")
plt.show()

sns.catplot(kind = 'box', data = file, x = 'Company', y = 'Calories')
plt.xticks(rotation = 90)
plt.title("Distribution of Calories by item from each company")
plt.show()

sns.catplot(kind = 'box', data = file, x = 'Company', y = 'Sodium \n(mg)')
plt.xticks(rotation = 90)
plt.title("Distribution of Sodium by item from each company")
plt.show()

sns.catplot(kind = 'box', data = file, x = 'Company', y = 'Fiber\n(g)')
plt.xticks(rotation = 90)
plt.title("Distribution of Fiber by item from each company")
plt.show()

In [ ]:
''' 
# but I also want to know about how many items are of different levels of calories, like sauces with very low calories, perhaps. I might use these later.
xlow_cal = file[file['Calories'] <= 50]
low_cal = file[(file['Calories'] <= 150)  & (file['Calories'] > 50)]
med_cal = file[(file['Calories'] <= 450)  & (file['Calories'] > 150)]
high_cal = file[file['Calories'] > 450]
listy = [xlow_cal, low_cal, med_cal, high_cal]
title_list=['<50', '50-150', '150-450', '>450']

for ii,i in enumerate(listy):
    corr = i.corr()
    sns.heatmap(corr, annot = True, linewidth = 0.5,fmt = '0.2f')
    plt.title('Correlations of {} calorie items'.format(title_list[ii]))
    plt.show()
'''

print('Uncomment code, or run directly by highlight --> ctrl+shift+enter for a lot of heatmaps across the various categories of calorie content')

# Correlation visualizations

Let's see how the columns correlate with each other. This will also be useful for any downstream machine learning analysis, as we will be able to drop redundant columns.

In [ ]:

# Let's see if we can predict company from the characteristics of the items:
# First let's find highly correlated data and drop redundant/correlated columns
corr = file.corr()
sns.heatmap(corr, annot = True, linewidth = 0.5, fmt = '0.2f')
plt.title('Correlations of untransformed data, all columns')
plt.show()


# Almost all of the columns values are positively correlated (if weakly) with each other, except sugar
# Now, some correlation comparisons:

# Is the weight Watchers column just a proxy for total calories? will drop NaN rows
file_WW_comp = file.dropna(subset = 'Weight Watchers\nPnts')

sns.lmplot(data = file_WW_comp, x = 'Weight Watchers\nPnts', y = 'Calories',line_kws = {'color':'r'})
plt.title('Calories vs. Weight Watcher Points')
plt.show()

# Fat vs. Sugar
sns.lmplot(data=file, x = 'Sugars\n(g)', y = 'Total Fat\n(g)', line_kws={'color':'r'})
plt.title('Grams of Fat vs. Sugar')
plt.show()




# Conclusion of correlation visualizations:
We can see that most measured values in the dataset correlate with each other positively, except sugar.
Interestingly, when we look at the scatter plot of sugar vs. fat quantities, we can see that the negative correlation is driven by a large number of items that are either predominantly fat with no sugar, or vice versa, seen by the points tightly hugging the x and y axes in the final figure of this section.

Another point of interest is that the "Weight Watchers Points" is extremely tightly correlated with total calories (correlation = 0.99), perhaps indicating that the Weight Watcher Points system is not contributing additional information about fast food items beyond total calorie count.



# Can we predict the company from the item characteristics?

Perhaps we can use machine learning to train an algorithm to identify the company name from the nutrient characteristics of the item.

In [ ]:
# Will use the cubed-root transformed data for this
# All the fat columns & Cholesterol are highly correlated, drop all except Total Fat
file_drop = file_re_trans.drop(['Saturated Fat\n(g)', 'Calories from\nFat', 'Cholesterol\n(mg)'], axis = 1)

# Weight Watcher points correlates extremely closely with total calories, also was imputed for half of the companies in the dataset,: also drop
file_drop = file_drop.drop(['Weight Watchers\nPnts'], axis = 1)

# Let's do this only for the items > 50 Calories:
file_drop = file_drop[file_drop['Calories'] > (50 ** (1/3))]

# Set random state for reproducibility
SEED = 656

# Prepare goal and predictive variables, and split data set into training and test portions:
X = file_drop.drop(['Item', 'Company'], axis = 1)

y = file_drop['Company']
y = y.factorize()
y = y[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = SEED)

# let's see if we can identify companies from their items
# Set up XGBoost and GridSearch algorithms:
sc = StandardScaler()
xg = xgb.XGBClassifier(objective = "reg:linear", random_state=SEED)
param = {"max_depth":[10, 20, 30],'n_estimators':[10,20,30],'gamma':[0.05, 0.15, 0.3]}
cv = GridSearchCV(xg, param_grid = param, cv = 5)

# Fit data to algorithms and test
sc.fit_transform(X_train)
sc.transform(X_test)
model = cv.fit(X_train, y_train)
best = model.best_estimator_
best_param = model.best_params_
score = best.score(X_test, y_test)
print('For this model, I dropped items with total calories <50, dropped a number of redundant columns, and used the cubed-root transformed data. \n')
accuracy_if_guessing = file['Company'].value_counts().max() / len(file)


print('Best Scoring Model Accuracy:\n', score, '\n', f"\nAccuracy if guessing company using the mode (always guess McDonald's): {accuracy_if_guessing}", '\n\n', 'Best Tested Parameters of the XGBoost model:', best_param)

Not very good at predicting the company, with only ~63% accuracy! 

However this is still notably higher than the accuracy (31%) you might expect by chance (if you only guessed the most common company in the dataset -- McDonald's -- for each item). 